**Final Results:**

```
Numpy CPU: 33.72 s 
Numpy GPU: 22.60 s 
Torch CPU: 24.91 s 
Torch GPU:  4.57 s
CuPy GPU:   8.36 s
```



In [ ]:
import numpy as np
import torch
from timeit import timeit

TOTAL_TIME = 1 #s
DT = 0.001
POPULATION_SIZE = 1000
BASE_CURRENT = 10000
CONNECTION_CHANCE = 1
REPEAT_TEST = 10

In [ ]:
if (torch.cuda.is_available()):
    DEVICE = 'cuda'
    workers = 1
else:
    DEVICE = 'cpu'
    workers = 0
print(f'Device is set to {DEVICE}\nNumber of workers: {workers}')

Device is set to cuda
Number of workers: 1


# Numpy

In [ ]:
import numpy as np

class Stimulus:
    def __init__(self, dt, output, neurons):
        self.output = output
        self.neurons = neurons
        self.dt = dt

    def __call__(self, timestep):
        return self.output(timestep * self.dt)


class NeuronGroup:
    def __init__(self, dt, population_size, connection_chance, total_time, stimuli = set(),
    neuron_type = "LIF", **kwargs):
        self.dt = dt
        self.N = population_size
        self.total_timepoints = int(total_time/dt)
        self.kwargs = kwargs
        self.connection_chance = connection_chance
        self.base_current = kwargs.get('base_current', 1E-9)
        self.u_thresh = kwargs.get('u_thresh', 35E-3)
        self.u_rest = kwargs.get('u_rest', -63E-3)
        self.refractory_timepoints = kwargs.get('tau_refractory', 0.002) / self.dt
        self.excitatory_chance = kwargs.get('excitatory_chance',  0.8)
        self.refractory = np.ones((self.N,1)) * self.refractory_timepoints
        self.current = np.zeros((self.N,1))
        self.potential = np.ones((self.N,1)) * self.u_rest
        self.spike_train = np.zeros((self.N, self.total_timepoints), dtype= np.bool)
        self.weights = np.random.rand(self.N,self.N)
        np.fill_diagonal(self.weights, 0)
        self.connections = (np.random.rand(*self.weights.shape) + self.connection_chance).astype(np.int)
        self.excitatory_neurons = (np.random.rand(*self.weights.shape) + self.excitatory_chance).astype(np.int) * 2 -1 
        self.AdjacencyMatrix = self.connections * self.excitatory_neurons * self.weights
        self.stimuli = np.array(list(stimuli))
        self.StimuliAdjacency = np.zeros((self.N, len(stimuli)), dtype=np.bool)
        for i, stimulus in enumerate(self.stimuli):
            self.StimuliAdjacency[stimulus.neurons, i] = True

    def LIF(self):
        """
        Leaky Integrate-and-Fire Neural Model
        """
        Rm = self.kwargs.get("Rm", 135E6)
        Cm = self.kwargs.get("Cm", 14.7E-12)
        tau_m = Rm*Cm
        exp_term = np.exp(-self.dt/tau_m)
        u_base = (1-exp_term) * self.u_rest
        return u_base + exp_term * self.potential + self.current*self.dt/Cm 

    def get_stimuli_current(self):
        call_stimuli =  np.vectorize(lambda stim: stim(self.timepoint))
        stimuli_output = call_stimuli(self.stimuli)
        stimuli_current = (stimuli_output * self.StimuliAdjacency).sum(axis = 1)
        return stimuli_current.reshape(self.N,1)
    
    def run(self):
        for self.timepoint in range(self.total_timepoints):
            ### LIF update
            self.refractory +=1
            self.potential = self.LIF()
            ### Reset currents
            self.current = np.zeros((self.N,1)) 
            ### Spikes 
            spikes = self.potential>self.u_thresh
            self.spike_train[:,self.timepoint] = spikes.ravel()
            self.refractory *= np.logical_not(spikes)
            ### Transfer currents + external sources
            new_currents = (spikes * self.AdjacencyMatrix).sum(axis = 0).reshape(self.N,1) * self.base_current
            open_neurons = self.refractory >= self.refractory_timepoints
            self.current += new_currents * open_neurons
            self.current += self.get_stimuli_current() #instead of this we can use :
            # self.current = np.sum(self.stimuli_adjancy * self.stimuli_current,axis = 1, keepdims = True) 
            

    def _spike_train_repr(self, spike_train):
        string = ''
        for t in spike_train:
            string += '|' if t else ' '
        return string

    def display_spikes(self):
        spike_train_display = ' id\n' + '=' * 5 + '╔' + '═' * self.total_timepoints + '╗\n'
        for i, spike_train in enumerate(self.spike_train):
            spike_train_display += str(i) + ' ' * (5 - len(str(i))) \
            + '║' + self._spike_train_repr(spike_train) + '║\n'  
        spike_train_display +=' ' * 5 + '╚' + '═' * self.total_timepoints + '╝'
        print(spike_train_display)


In [ ]:
stimuli = {Stimulus(0.001, lambda t: 2E-9, [0,1]),
           Stimulus(0.001, lambda t: 2E-9 * t, [2]),
           Stimulus(0.001, lambda t: 2E-9 * np.sin(500*t), [3])}
G = NeuronGroup(dt = DT, 
                population_size = POPULATION_SIZE, 
                connection_chance = CONNECTION_CHANCE,
                total_time = TOTAL_TIME,
                base_current = 2E-9,
                stimuli = stimuli)

vectorized_time = timeit('G.run()', number = REPEAT_TEST, globals=globals())

In [ ]:
print("CPU numpy time: ", vectorized_time)

CPU numpy time:  22.391187097


In [ ]:
print("GPU numpy time: ", vectorized_time)

GPU numpy time:  22.391187097


# Torch

In [ ]:


class Stimulus:
    def __init__(self, dt, output, neurons):
        self.output = output
        self.neurons = neurons
        self.dt = dt

    def __call__(self, timestep):
        return self.output(timestep * self.dt)


class NeuronGroup:
    def __init__(self, dt, population_size, connection_chance, total_time, stimuli = set(),
    neuron_type = "LIF", **kwargs):
        self.dt = dt
        self.N = population_size
        self.total_timepoints = int(total_time/dt)
        self.stimuli = stimuli
        self.kwargs = kwargs
        self.connection_chance = connection_chance
        self.base_current = kwargs.get('base_current', 1E-9)
        self.u_thresh = kwargs.get('u_thresh', 35E-3)
        self.u_rest = kwargs.get('u_rest', -63E-3)
        self.refractory_timepoints = kwargs.get('tau_refractory', 0.002) / self.dt
        self.excitatory_chance = kwargs.get('excitatory_chance',  0.8)
        self.refractory = torch.ones((self.N,1)).to(DEVICE) * self.refractory_timepoints
        self.current = torch.zeros((self.N,1)).to(DEVICE)
        self.potential = torch.ones((self.N,1)).to(DEVICE) * self.u_rest
        self.spike_train = torch.zeros((self.N, self.total_timepoints), dtype= torch.bool).to(DEVICE)
        self.weights = np.random.rand(self.N,self.N)
        np.fill_diagonal(self.weights, 0)
        self.weights = torch.from_numpy(self.weights)
        self.connections = (torch.rand(*self.weights.shape) + self.connection_chance).type(torch.int)
        self.excitatory_neurons = (torch.rand(*self.weights.shape) + self.excitatory_chance).type(torch.int) * 2 -1
        self.AdjacencyMatrix = self.connections * self.excitatory_neurons * self.weights
        self.AdjacencyMatrix = self.AdjacencyMatrix.to(DEVICE)
        self.stimuli = np.array(list(stimuli))
        self.StimuliAdjacency = np.zeros((self.N, len(stimuli)), dtype=np.bool)
        for i, stimulus in enumerate(self.stimuli):
            self.StimuliAdjacency[stimulus.neurons, i] = True

    def LIF(self):
        """
        Leaky Integrate-and-Fire Neural Model
        """
        Rm = self.kwargs.get("Rm", 135E6)
        Cm = self.kwargs.get("Cm", 14.7E-12)
        tau_m = Rm*Cm
        exp_term = torch.exp(torch.tensor(-self.dt/tau_m))
        u_base = (1-exp_term) * self.u_rest
        new_potential = u_base + exp_term * self.potential + self.current*self.dt/Cm
        return new_potential.to(DEVICE)

    def get_stimuli_current(self):
        call_stimuli =  np.vectorize(lambda stim: stim(self.timepoint))
        stimuli_output = call_stimuli(self.stimuli)
        stimuli_current = (stimuli_output * self.StimuliAdjacency).sum(axis = 1)
        return torch.tensor(stimuli_current.reshape(self.N,1)).to(DEVICE)
    
    def run(self):
        for self.timepoint in range(self.total_timepoints):
            ### LIF update
            self.refractory +=1
            self.potential = self.LIF()
            ### Reset currents
            self.current = torch.zeros((self.N,1)).to(DEVICE)
            ### Spikes 
            spikes = self.potential>self.u_thresh
            self.spike_train[:,self.timepoint] = spikes.ravel()
            self.refractory *= torch.logical_not(spikes).to(DEVICE)
            ### Transfer currents + external sources
            new_currents = (spikes * self.AdjacencyMatrix).sum(axis = 0).reshape(self.N,1) * self.base_current
            open_neurons = self.refractory >= self.refractory_timepoints
            self.current += new_currents * open_neurons
            self.current += self.get_stimuli_current()

    def _spike_train_repr(self, spike_train):
        string = ''
        for t in spike_train:
            string += '|' if t else ' '
        return string

    def display_spikes(self):
        spike_train_display = ' id\n' + '=' * 5 + '╔' + '═' * self.total_timepoints + '╗\n'
        for i, spike_train in enumerate(self.spike_train):
            spike_train_display += str(i) + ' ' * (5 - len(str(i))) \
            + '║' + self._spike_train_repr(spike_train) + '║\n'  
        spike_train_display +=' ' * 5 + '╚' + '═' * self.total_timepoints + '╝'
        print(spike_train_display)


In [ ]:
stimuli = {Stimulus(0.001, lambda t: 2E-9, [0,1]),
           Stimulus(0.001, lambda t: 2E-9 * t, [2]),
           Stimulus(0.001, lambda t: 2E-9 * np.sin(500*t), [3])}
G = NeuronGroup(dt = DT, 
                population_size = POPULATION_SIZE, 
                connection_chance = CONNECTION_CHANCE,
                total_time = TOTAL_TIME,
                base_current = 2E-9,
                stimuli = stimuli)

vectorized_time = timeit('G.run()', number = REPEAT_TEST, globals=globals())

In [ ]:
print("CPU torch time: ", vectorized_time)

CPU torch time:  5.146830141999999


In [ ]:
print("GPU torch time: ", vectorized_time)

GPU torch time:  5.146830141999999


## Learning


In [ ]:
class RFSTDP:
    def __init__(self, NeuronGroup,
                 interval_time = 0.001, # seconds
                 pre_post_rate = 0.001,
                 reward_pre_post_rate = 0.002,
                 post_pre_rate = -0.001,
                 reward_post_pre_rate = 0.001,
                 ):
        """
        Reward-modulated Flat STDP 
        """
        self.NeuronGroup = NeuronGroup
        self.AdjacencyMatrix = NeuronGroup.AdjacencyMatrix
        self.interval_timepoints = int(interval_time / NeuronGroup.dt) #timepoints
        self.total_timepoints = NeuronGroup.total_timepoints
        self.spike_train = NeuronGroup.spike_train
        self.reward_based = True
        self.pre_post_rate = pre_post_rate
        self.post_pre_rate = post_pre_rate
        self.reward_pre_post_rate = reward_pre_post_rate
        self.reward_post_pre_rate = reward_post_pre_rate

    def __call__(self, reward):
        spike_train = self.NeuronGroup.spike_train
        padded_spike_train = torch.nn.functional.pad(spike_train,
            (self.interval_timepoints, self.interval_timepoints, 0, 0),
             mode='constant', value=0)
        for i in range(self.total_timepoints + self.interval_timepoints):
            section = padded_spike_train[:,i:i+self.interval_timepoints]
            span = section.sum(axis = 1).type(torch.bool)
            first = padded_spike_train[:,i]
            last = padded_spike_train[:,i+self.interval_timepoints]
            if reward:
                self.AdjacencyMatrix += self.reward_pre_post_rate * (first * span * self.AdjacencyMatrix)
                self.AdjacencyMatrix += self.reward_post_pre_rate * (span  * last * self.AdjacencyMatrix)
            if not reward:
                self.AdjacencyMatrix += self.pre_post_rate * (first * span * self.AdjacencyMatrix)
                self.AdjacencyMatrix += self.post_pre_rate * (span  * last * self.AdjacencyMatrix)


In [ ]:
stimuli = {Stimulus(0.001, lambda t: 2E-9, [0,1]),
        #    Stimulus(0.001, lambda t: 2E-9 * t, [2]),
        #    Stimulus(0.001, lambda t: 2E-9 * np.sin(500*t), [3])
           }
G = NeuronGroup(dt = 0.001, 
                population_size = 5, 
                connection_chance = CONNECTION_CHANCE,
                total_time = 1,
                base_current = 2E-9,
                stimuli = stimuli,
                tau_refractory = 0.002)
G.run()
print(G.AdjacencyMatrix)
learning = RFSTDP(G)
learning(True)
print(G.AdjacencyMatrix)

tensor([[ 0.0000,  0.6871,  0.4502,  0.6704,  0.0838],
        [-0.4723,  0.0000,  0.9621,  0.6461,  0.9022],
        [ 0.8080,  0.9272,  0.0000, -0.3516,  0.5366],
        [ 0.0464,  0.2355, -0.4373,  0.0000,  0.7221],
        [ 0.2028,  0.7988,  0.5973,  0.4617,  0.0000]], device='cuda:0',
       dtype=torch.float64)
tensor([[ 0.0000, 13.7124,  1.9033,  2.8259,  0.3528],
        [-9.4247,  0.0000,  4.0680,  2.7233,  3.7991],
        [16.1236, 18.5036,  0.0000, -1.4821,  2.2597],
        [ 0.9257,  4.7004, -1.8488,  0.0000,  3.0409],
        [ 4.0464, 15.9403,  2.5253,  1.9462,  0.0000]], device='cuda:0',
       dtype=torch.float64)


In [ ]:
G.display_spikes()

 id
=====╔══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [ ]:
G.run()
G.display_spikes()

 id
=====╔══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

# CuPy

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!pip install cupy-cuda110

     |████████████████████████████████| 165.3MB 35kB/s 


In [ ]:
import cupy as cp
import numpy as np


class Stimulus:
    def __init__(self, dt, output, neurons):
        self.output = output
        self.neurons = neurons
        self.dt = dt

    def __call__(self, timestep):
        return self.output(timestep * self.dt)


class NeuronGroup:
    def __init__(self, dt, population_size, connection_chance, total_time, stimuli = set(),
    neuron_type = "LIF", **kwargs):
        self.dt = dt
        self.N = population_size
        self.total_timepoints = int(total_time/dt)
        self.kwargs = kwargs
        self.connection_chance = connection_chance
        self.base_current = kwargs.get('base_current', 1E-9)
        self.u_thresh = kwargs.get('u_thresh', 35E-3)
        self.u_rest = kwargs.get('u_rest', -63E-3)
        self.refractory_timepoints = kwargs.get('tau_refractory', 0.002) / self.dt
        self.excitatory_chance = kwargs.get('excitatory_chance',  0.8)
        self.refractory = cp.ones((self.N,1), dtype = cp.uint8) * self.refractory_timepoints
        self.current = cp.zeros((self.N,1), dtype = cp.float64)
        self.potential = cp.ones((self.N,1), dtype = cp.float64) * self.u_rest
        self.spike_train = cp.zeros((self.N, self.total_timepoints), dtype= cp.bool)
        self.weights = cp.random.rand(self.N,self.N)
        cp.fill_diagonal(self.weights, 0)
        self.connections = (cp.random.rand(*self.weights.shape) + self.connection_chance).astype(cp.int)
        self.excitatory_neurons = (cp.random.rand(*self.weights.shape) + self.excitatory_chance).astype(cp.int) * 2 -1 
        self.AdjacencyMatrix = self.connections * self.excitatory_neurons * self.weights
        self.stimuli = np.array(list(stimuli))
        self.StimuliAdjacency = np.zeros((self.N, len(stimuli)), dtype=np.bool)
        for i, stimulus in enumerate(self.stimuli):
            self.StimuliAdjacency[stimulus.neurons, i] = True

    def LIF(self):
        """
        Leaky Integrate-and-Fire Neural Model
        """
        Rm = self.kwargs.get("Rm", 135E6)
        Cm = self.kwargs.get("Cm", 14.7E-12)
        tau_m = Rm*Cm
        exp_term = cp.exp(-self.dt/tau_m)
        u_base = (1-exp_term) * self.u_rest
        return u_base + exp_term * self.potential + self.current*self.dt/Cm 

    def get_stimuli_current(self):
        call_stimuli =  np.vectorize(lambda stim: stim(self.timepoint))
        stimuli_output = call_stimuli(self.stimuli)
        stimuli_current = (stimuli_output * self.StimuliAdjacency).sum(axis = 1)
        return cp.asarray(stimuli_current.reshape(self.N,1))
    
    def run(self):
        for self.timepoint in range(self.total_timepoints):
            ### LIF update
            self.refractory +=1
            self.potential = self.LIF()
            ### Reset currents
            self.current = cp.zeros((self.N,1)) 
            ### Spikes 
            spikes = self.potential>self.u_thresh
            self.spike_train[:,self.timepoint] = spikes.ravel()
            self.refractory *= cp.logical_not(spikes)
            ### Transfer currents + external sources
            new_currents = (spikes * self.AdjacencyMatrix).sum(axis = 0).reshape(self.N,1) * self.base_current
            open_neurons = self.refractory >= self.refractory_timepoints
            self.current += new_currents * open_neurons
            self.current += self.get_stimuli_current() #instead of this we can use :
            # self.current = cp.sum(self.stimuli_adjancy * self.stimuli_current,axis = 1, keepdims = True) 
            

    def _spike_train_repr(self, spike_train):
        string = ''
        for t in spike_train:
            string += '|' if t else ' '
        return string

    def display_spikes(self):
        spike_train_display = ' id\n' + '=' * 5 + '╔' + '═' * self.total_timepoints + '╗\n'
        for i, spike_train in enumerate(self.spike_train):
            spike_train_display += str(i) + ' ' * (5 - len(str(i))) \
            + '║' + self._spike_train_repr(spike_train) + '║\n'  
        spike_train_display +=' ' * 5 + '╚' + '═' * self.total_timepoints + '╝'
        print(spike_train_display)


In [ ]:
stimuli = {Stimulus(0.001, lambda t: 2E-9, [0,1]),
           Stimulus(0.001, lambda t: 2E-9 * t, [2]),
           Stimulus(0.001, lambda t: 2E-9 * cp.sin(500*t), [3])}
G = NeuronGroup(dt = DT, 
                population_size = POPULATION_SIZE, 
                connection_chance = CONNECTION_CHANCE,
                total_time = TOTAL_TIME,
                base_current = 2E-9,
                stimuli = stimuli)

vectorized_time = timeit('G.run()', number = REPEAT_TEST, globals=globals())

In [ ]:
print("GPU CuPy time: ", vectorized_time)

GPU CuPy time:  9.690362012000008
